In [35]:
import pandas as pd
import numpy as np

In [36]:
df_train = pd.read_csv('train.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [37]:
df_1 = df_train.drop(['Survived','Name','SibSp','Parch','Ticket','Cabin'],axis='columns')
df_1['Age'] = df_1.groupby(['Pclass', 'Sex'])['Age'] \
              .transform(lambda x: x.fillna(x.median()))

df_1.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked
0,1,3,male,22.0,7.2500,S
1,2,1,female,38.0,71.2833,C
2,3,3,female,26.0,7.9250,S
3,4,1,female,35.0,53.1000,S
4,5,3,male,35.0,8.0500,S


In [42]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

ct = ColumnTransformer(transformers = [('Sex',OneHotEncoder(drop='first'),['Sex']),('Port',OneHotEncoder(drop='first'),['Embarked']),('num', numeric_transformer,['Age','Fare'])],remainder='passthrough')
df = ct.fit_transform(df_1)
df,

(array([[ 1.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         -5.02445171e-01,  1.00000000e+00,  3.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          7.86845294e-01,  2.00000000e+00,  1.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         -4.88854258e-01,  3.00000000e+00,  3.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         -1.76263239e-01,  8.89000000e+02,  3.00000000e+00],
        [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -4.43810379e-02,  8.90000000e+02,  1.00000000e+00],
        [ 1.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         -4.92377828e-01,  8.91000000e+02,  3.00000000e+00]]),)

In [43]:
from sklearn.linear_model import LogisticRegression

In [44]:
model = LogisticRegression(max_iter=1000)
model.fit(df,df_train['Survived'])

LogisticRegression(max_iter=1000)

In [47]:
model.score(df,df_train['Survived'])

0.797979797979798

In [49]:
test = pd.read_csv('test.csv')

In [51]:
test_1 = test.drop(['Name','SibSp','Parch','Ticket','Cabin'],axis='columns')
test_1.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked
0,892,3,male,34.5,7.8292,Q
1,893,3,female,47.0,7.0000,S
2,894,2,male,62.0,9.6875,Q
3,895,3,male,27.0,8.6625,S
4,896,3,female,22.0,12.2875,S


In [52]:
X_test = ct.transform(test_1)
X_test

array([[ 1.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
        -4.90783161e-01,  8.92000000e+02,  3.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
        -5.07478843e-01,  8.93000000e+02,  3.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
        -4.53366871e-01,  8.94000000e+02,  2.00000000e+00],
       ...,
       [ 1.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
        -5.02445171e-01,  1.30700000e+03,  3.00000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
        -4.86337422e-01,  1.30800000e+03,  3.00000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.98244277e-01,  1.30900000e+03,  3.00000000e+00]])

In [53]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [54]:
 submission = pd.DataFrame({
    'PassengerId' : test['PassengerId'],
    'Survived' : y_pred})
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [55]:
submission.to_csv('submission.csv', index=False)

In [56]:
from sklearn.metrics import classification_report

print(classification_report(df_train['Survived'],model.predict(df) ))

              precision    recall  f1-score   support

           0       0.82      0.85      0.84       549
           1       0.75      0.71      0.73       342

    accuracy                           0.80       891
   macro avg       0.79      0.78      0.78       891
weighted avg       0.80      0.80      0.80       891

